In [5]:
import requests
from datetime import date
import pandas as pd
import itertools
import csv
import re
from unicodedata import normalize
from decouple import Config, RepositoryEnv

In [6]:
DOTENV_FILE = '.env'
config = Config(RepositoryEnv(DOTENV_FILE))

In [7]:
url_museos = config('URL_MUSEOS')
url_cines = config('URL_CINES')
url_bibliotecas = config('URL_BIBLIOTECAS')

In [8]:
def create_name(categoria):
    name = categoria + '\\' + date.today().strftime('%Y-%m') + '\\' + categoria + '-' + date.today().strftime('%d-%m-%Y') + '.csv'
    return name

In [9]:
name_museos = create_name('museos')
name_cines = create_name('cines')
name_bibliotecas = create_name('bibliotecas')

In [10]:

def normalize_string(s):
    s = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize('NFD', s), 0, re.I)
    return normalize('NFC', s).lower()

In [11]:
def change_header_csv(name):
    with open(name, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        header = list(map(normalize_string, header))
        with open(name, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(reader)

In [16]:
def get_csv(url, name):
    r = requests.get(url)
    with open(name, 'wb') as f:
        f.write(r.content)
    change_header_csv(name)

In [13]:
def download_data():
    get_csv(url_museos, name_museos)
    get_csv(url_cines, name_cines)
    get_csv(url_bibliotecas, name_bibliotecas)

In [39]:
def changeHeader(df):
    df.rename({'cod_loc': 'cod_localidad', 'idprovincia': 'id_provincia', 'iddepartamento': 'id_departamento', 'cp': 'cod_postal', 'telefono': 'numero_telefono'}, axis=1, inplace=True)
    return df

In [40]:
def create_table(museos, cines, bibliotecas):
    df_museos = pd.read_csv(museos)
    df_cines = pd.read_csv(cines)
    df_bibliotecas = pd.read_csv(bibliotecas)
    headersBiliotecas = ['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'cp', 'telefono', 'mail', 'web','domicilio']
    headers = ['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre','cp', 'telefono', 'mail', 'web']
    
    df_museos = df_museos[headers]
    df_cines = df_cines[headers]
    df_bibliotecas = df_bibliotecas[headersBiliotecas]

    #Merge the dataframes
    df_normalize = pd.concat([df_museos, df_cines, df_bibliotecas])
    return changeHeader(df_normalize)

In [41]:
create_table(name_museos,name_cines,name_bibliotecas)

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,cod_postal,numero_telefono,mail,web,domicilio
0,6588100,6.0,6588.0,Espacios de Exhibición Patrimonial,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,B6500EVL,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,NaN
1,6077010,6.0,6077.0,Espacios de Exhibición Patrimonial,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,B2740FMJ,452 931,museoarrecifes@hotmail.com,NaN,NaN
2,6035010,6.0,6035.0,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,S3561AKT,481 200,NaN,NaN,NaN
3,6035010,6.0,6035.0,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,B1870CBC,4205 9567,NaN,NaN,NaN
4,6049020,6.0,6049.0,Espacios de Exhibición Patrimonial,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,B7300XAA,434 811,museoazul@yahoo.com,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
25,6287010,6.0,6287.0,Bibliotecas Populares,Buenos Aires,General Alvear,Biblioteca Popular Florencio Balcarce,7236,481246,bibliofbalcarce@yahoo.com.ar,NaN,San Martín 341
26,6595060,6.0,6595.0,Bibliotecas Populares,Buenos Aires,Hinojo,Biblioteca Popular Domingo Faustino Sarmiento,7318,s/d,s/d,NaN,Av.Mitre s/n
27,6413050,6.0,6413.0,Bibliotecas Populares,Buenos Aires,Junín,Biblioteca Popular Olegario V. Andrade,B600GZC,428071,olegan@ciudad.com.ar,NaN,Belgrano 191
28,6434010,6.0,6434.0,Bibliotecas Populares,Buenos Aires,Lanús Este,Biblioteca Popular Libertador San Martín,B1824ICK,42493631,libertadorsanmartin.bib_lanus@yahoo.com.ar,NaN,Eva Perón 2597
